In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import re
import pandas as pd
import numpy as np
import time
import io
from bs4 import BeautifulSoup 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving notincomb.csv to notincomb.csv


In [ ]:
#df_movie_final = pd.read_csv(io.BytesIO(uploaded['data_moviefinal.csv']))
df_movie_final = pd.read_csv('notincomb.csv')

In [ ]:
df_movie_final.shape

(12, 7)

In [ ]:
df_movie_final.head()

,ID,movie_title,class,year,runtime,rating,meta
0,tt8196692,Soul Hunters,NaN,2019.0,101.0,6.8,NaN
1,tt0179196,"Goodbye, 20th Century",NaN,1998.0,83.0,6.8,NaN
2,tt6871448,Lollos 7 Laat die bene bewe,NaN,2015.0,77.0,NaN,NaN
3,tt0114011,Now and Then,PG-13,1995.0,100.0,6.8,50.0
4,tt8170902,Qué León,PG-13,2018.0,93.0,6.2,NaN


In [ ]:
data_final = []
num_range = df_movie_final.shape[0]
for i in range(0,num_range):
    
    url = "https://www.imdb.com/title/" + df_movie_final.ID[i]
    Id = df_movie_final.ID[i]
    
    #intialize variables
    poster      = ''
    budget      = ''
    genre       = '' 
    Director    = ''
    language    = ''
    Keywords    = '' 
    tagl        = ''
    mpaac       = ''
    budget      = ''
    revenue     = ''
    description = ''
    Actors      = ''
    writer      = ''
    ratingCount = ''
    company     = ''
    genres      = []
    

    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')

    # Poster Link
    poster = soup.find('div',{'class': 'poster'})
    if not(poster == None):   
        poster = poster.find('img')
        poster = poster.get('src')

    # Movie Summary 
    for i in soup.find_all('div',{'class': 'summary_text'}):
        i = i.text
        description = i.strip()

    #Director , writer and actors of the movie
    plot_summaryDirector = soup.find('div',{'class': 'plot_summary'})
    Director = []
    Actors   = []
    writer   = [] 
    if not(plot_summaryDirector == None):
      for i in plot_summaryDirector.find_all('div',{'class': 'credit_summary_item'}):
          i = i.text
          i = i.strip()
          if "Director:" in i:
              Director = i[10:]
              Director = Director.split(',')
              Director = Director
          elif "Directors:" in i:
              Director = i[11:]
              Director = Director.split(',')
              Director = Director
          elif "Writer:" in i:
              writer = i[8:]
              writer = writer.split(',')
              writer = writer
          elif "Writers:" in i:
              writer = i[9:]
              writer = writer.split(',')
              writer = writer
          elif "Stars:" in i:
              Actors = i[6:]
              Actors = Actors.split('|')
              Actors = Actors[0:]
              s      = ([s.strip() for s in Actors])
              Actors =([x for x in s if x])

    # rating Count of the movie 
    ratingCount = soup.find("span", {"itemprop" : "ratingCount"})
    if not(ratingCount == None):
        ratingCount = ratingCount.string
    
    #language list
    txtblock = soup.find('div',{'id': 'titleDetails'}) 
    if not(txtblock == None):
      for i in txtblock.find_all('div',{'class': 'txt-block'}):
          i = i.text
          i = i.strip()
          if "Language:" in i:
              language = i.split()
              language = language[1:]
              s        = ([s.strip('|') for s in language])
              language =([x for x in s if x])

    text1 = soup.find('div',{'id': 'titleStoryLine'})

    # keywords of the movie
    if not(text1 == None):    
      for i in text1.find_all('div',{'class': 'see-more inline canwrap'}):
          i = i.text
          i = i.strip()
          if "Keywords:" in i:
              Keywords = i.split('|')
              Keywords = Keywords[1:]
              s        = ([s.strip() for s in Keywords])
              Keywords =([x for x in s if x])	
          elif "Genres:" in i:
              genres = i.split(':')
              genres = genres[1:]
              s = ([s.strip() for s in genres])
              genres = ([x for x in s if x])	
              genre = genres

    #tagline of the movie and motion picture Rating 
    if not(text1 == None):
      for i in text1.find_all('div',{'class': 'txt-block'}):
          i = i.text
          i = i.strip()
          if "Taglines" in i:
             tagl = i
             tagl = tagl[10:]
          elif "MPAA" in i:
             mpaac = i.split('|')
             mpaac = mpaac[0:]


    text2 = soup.find('div',{'id': 'titleDetails'})

    # Budget , Revenue and the production company of the movie
    if not(text2 == None):
      for i in text2.find_all('div',{'class': 'txt-block'}):
          i = i.text
          i = i.strip()
          if "Budget:" in i:
              budget = i.split(':')
              budget = budget[1:]
              s      = ([s.strip() for s in budget])
              budget =([x for x in s if x])
          elif "Cumulative Worldwide Gross" in i: 
              revenue = i.split(':')
              revenue = revenue[1:]
          elif "Production Co" in i:
              company = i.split(':')
              company = company[1:]
              s       = ([s.strip() for s in company])
              company =([x for x in s if x])	


    data1 = {
        "ID"                : Id,
        "Poster"            : poster,
        "Genre"             : genre,    
        "Director"          : Director,
        "Language"          : language,
        "Keywords"          : Keywords,
        "Tagline"           : tagl,
        "MPAA"              : mpaac,
        "Budget"            : budget,
        "revenue"           : revenue,
        "Summary"           : description,
        "Actors"            : Actors,
        "writer"            : writer,
        "No_of_Votes"       : ratingCount,
        "Production_Company": company
        }
    data_final.append(data1)

In [ ]:
df = pd.DataFrame(data_final)

In [ ]:
df.shape

(12, 15)

In [ ]:
df.to_csv('data_moviefinal_12.csv',index=False)

In [ ]:
from google.colab import files
files.download('data_moviefinal_12.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.head()

,ID,Poster,Genre,Director,Language,Keywords,Tagline,MPAA,Budget,revenue,Summary,Actors,writer,No_of_Votes,Production_Company
0,tt3365338,https://m.media-amazon.com/images/M/MV5BMTY0NT...,[Horror],[Kenneth Guertin],[English],[See All (1) »],,,,,"In this gripping tale of terror, twenty someth...","[Jackson Kuehn, Mary Elizabeth Boylan, Chad Bi...","[Kenneth Guertin (story), Kenneth Guertin |\n...",231,"[Make Believe Entertainment, Wired Reel Entert..."
1,tt0219908,https://m.media-amazon.com/images/M/MV5BYzQwYz...,[Horror],[Andrea Bianchi (as Frank Drew White)],[English],"[slasher, maniac, death cult, prostitutes, See...",,,,,A murderous cult kidnaps French prostitutes an...,"[Bo Svenson, Chuck Connors, Robert Ginty, See ...",[Georges Friedland (screenplay and dialogue) (...,80,[Eurociné \nSee more »]
2,tt0238302,https://m.media-amazon.com/images/M/MV5BZjQzMW...,[Horror |\n Thriller],[Colin Eggleston],[English],"[slasher, australian giallo, giallo, prostitut...",Being terrified is just the beginning.,[Motion Picture Rating\n (M...,,,A woman living in Dallas discovers that her hu...,"[P.J. Soles, Kit Taylor, Grigor Taylor, See fu...",[Ron McLean (based on a screenplay by)],134,"[Crystal Film Corporation, Premiere Film Marke..."
3,tt0496231,https://m.media-amazon.com/images/M/MV5BMTc5Nj...,[Horror],[David Palmieri],[English],"[serial killer, maniac, backwood slasher, clow...",,,,,A 20-something camping trip goes horribly wron...,"[Cassie Daniels, Chris Todd, Jennifer Peo, See...","[Jeremiah Campbell (additional material), Kei...",81,
4,tt9614972,https://m.media-amazon.com/images/M/MV5BYjJjOW...,[Drama |\n Horror |\n Romance],[Caarthick Raju],[Tamil],,,,,,Add a Plot »,"[Poornima Bhagyaraj, Sundeep Kishan, Anya Singh]",[],None,
